In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Strategy1 : Relative Position<br>
Strategy2 : ROC<br>
Strategy3 : Proximity to Support<br>
Startegy4 : Minimum Trailing Volatility<br>
Strategy5 : Sector Rotation<br>
Strategyrf : ML Prediction Strategy<br>

Strategy1:<br>
    Select bottom5 stocks ranked by Relative Position and hold for length(=30) days.<br>
<br>
Strategy2:<br>
    Select top5 stocks ranked by ROC with ROC>0 and hold for length(=30) days.<br>
<br>
Strategy3:<br>
    Select bottom5 stocks ranked by Proximity to Support and hold for length(=30) days.<br>
<br>
Strategy4:<br>
    Select bottom5 stocks ranked by Volatility with Yearly returns > risk_free_rate and hold for length(=30) days.<br>
<br>
Strategy5:<br>
    Select top5 stocks ranked by ROC in sector which has highest ROC and hold for length(=30) days.<br>
<br>
Strategyrf:<br>
    1. Random Forest predicts which strategy will give best Net return over next length(=30) days.<br>
    2. Select stocks of that strategy and hold for length(=30) days.<br>

In [372]:
#stock list of nifty100 2019 may to avoid survivorship bias 
#backtesting starts from 2019

stock_list = ['ABB.NS', 'ACC.NS', 'ADANIPORTS.NS', 'AMBUJACEM.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'AUROPHARMA.NS', 'DMART.NS', 'AXISBANK.NS', 
              'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BHEL.NS', 'BPCL.NS', 
              'BHARTIARTL.NS', 'INFRATEL.NS', 'BIOCON.NS', 'BOSCHLTD.NS', 'BRITANNIA.NS', 'CADILAHC.NS', 'CIPLA.NS', 'COALINDIA.NS', 'COLPAL.NS', 
              'CONCOR.NS', 'DLF.NS', 'DABUR.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GAIL.NS', 'GICRE.NS', 'GODREJCP.NS', 'GRASIM.NS', 
              'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HAVELLS.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 
              'HINDZINC.NS', 'HDFC.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'ITC.NS', 'IBULHSGFIN.NS', 'IOC.NS', 'INDUSINDBK.NS', 'INFY.NS', 
              'INDIGO.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'L&TFH.NS', 'LT.NS', 'LUPIN.NS', 'MRF.NS', 'M&M.NS', 'MARICO.NS', 'MARUTI.NS', 'MOTHERSUMI.NS', 
              'NHPC.NS', 'NMDC.NS', 'NTPC.NS', 'ONGC.NS', 'OFSS.NS', 'PAGEIND.NS', 'PETRONET.NS', 'PIDILITIND.NS', 'PEL.NS', 'POWERGRID.NS', 'PGHH.NS', 
              'RELIANCE.NS', 'SBILIFE.NS', 'SHREECEM.NS', 'SRTRANSFIN.NS', 'SIEMENS.NS', 'SBIN.NS', 'SAIL.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATAMTRDVR.NS', 
              'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'NIACL.NS', 'TITAN.NS', 'UPL.NS', 'ULTRACEMCO.NS', 'UBL.NS', 'MCDOWELL-N.NS', 'VEDL.NS', 'IDEA.NS', 
              'WIPRO.NS', 'YESBANK.NS', 'ZEEL.NS']

start_date="2005-03-01"
end_date="2025-06-01"

data=yf.download(stock_list,start=start_date,end=end_date,interval="1d")
data=data[1:]

[*********************100%***********************]  101 of 101 completed

9 Failed downloads:
['TATAMTRDVR.NS', 'INFRATEL.NS', 'SRTRANSFIN.NS', 'L&TFH.NS', 'MCDOWELL-N.NS', 'CADILAHC.NS', 'IBULHSGFIN.NS', 'MOTHERSUMI.NS', 'HDFC.NS']: YFTzMissingError('possibly delisted; no timezone found')


In [373]:
data.head()

Price        Adj Close                                             \
Ticker     CADILAHC.NS HDFC.NS IBULHSGFIN.NS INFRATEL.NS L&TFH.NS   
Date                                                                
2005-03-02         NaN     NaN           NaN         NaN      NaN   
2005-03-03         NaN     NaN           NaN         NaN      NaN   
2005-03-04         NaN     NaN           NaN         NaN      NaN   
2005-03-07         NaN     NaN           NaN         NaN      NaN   
2005-03-08         NaN     NaN           NaN         NaN      NaN   

Price                                                                   Close  \
Ticker     MCDOWELL-N.NS MOTHERSUMI.NS SRTRANSFIN.NS TATAMTRDVR.NS     ABB.NS   
Date                                                                            
2005-03-02           NaN           NaN           NaN           NaN  40.819855   
2005-03-03           NaN           NaN           NaN           NaN  41.238663   
2005-03-04           NaN           NaN           NaN           NaN  41.444855   
2005-03-07           NaN           NaN           NaN           NaN  41.294605   
2005-03-08           NaN           NaN           NaN           NaN  41.753368   

Price       ...     Volume                                            \
Ticker      ...     TCS.NS TECHM.NS    TITAN.NS UBL.NS ULTRACEMCO.NS   
Date        ...                                                        
2005-03-02  ...  2658112.0      NaN   2833600.0    NaN       91386.0   
2005-03-03  ...  2891160.0      NaN   2204820.0    NaN      123055.0   
2005-03-04  ...  4017872.0      NaN   2080760.0    NaN       72226.0   
2005-03-07  ...  3658032.0      NaN  20617400.0    NaN      140545.0   
2005-03-08  ...  5027680.0      NaN   6174960.0    NaN       88686.0   

Price                                                                
Ticker        UPL.NS     VEDL.NS    WIPRO.NS YESBANK.NS     ZEEL.NS  
Date                                                                 
2005-03-02  510090.0  19135640.0   9503252.0        NaN   7898625.0  
2005-03-03  197895.0  11054020.0  12652906.0        NaN  10280932.0  
2005-03-04  113040.0  11779980.0  20141190.0        NaN   9320324.0  
2005-03-07  248985.0  28156060.0   6830950.0        NaN   2311638.0  
2005-03-08  713235.0  22770360.0  14425332.0        NaN   3389290.0  

[5 rows x 514 columns]

In [374]:
split_date='2019-06-01'# date after which backtesting begins
risk_free_rate=0.07
cost_rate=0.002 # transaction cost rate
length=30 # freqency of strategy selection in trading days

In [375]:
daily_rate= (1+risk_free_rate)**(1/252) -1

In [376]:
split_date=pd.to_datetime(split_date)

In [ ]:
df=data['Close']
df = df.mask(df <= 0, np.nan)
df.ffill(inplace=True)
df= df.dropna(axis=1)
stock_list=df.columns

In [378]:
df.head()

Ticker,ABB.NS,ACC.NS,AMBUJACEM.NS,ASHOKLEY.NS,ASIANPAINT.NS,AUROPHARMA.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJAJFINSV.NS,BAJAJHLDNG.NS,...,SUNPHARMA.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TITAN.NS,ULTRACEMCO.NS,UPL.NS,VEDL.NS,WIPRO.NS,ZEEL.NS
Date,,,,,,,,,,,,,,,,,,,,,
2005-03-02,40.819855,261.901306,40.099876,7.090164,28.802921,25.783337,39.470455,163.547745,1033.979492,650.166016,...,38.799053,87.493935,29.000359,120.599174,9.450497,334.632751,41.028000,11.740839,36.570621,35.051968
2005-03-03,41.238663,264.604736,40.527275,7.180104,28.635908,27.042212,39.409973,170.594131,1078.528320,678.178223,...,38.972015,90.428963,29.135681,122.499870,9.511654,332.984589,41.021942,11.749317,37.957485,35.376640
2005-03-04,41.444855,266.027496,40.513695,7.090164,28.110987,28.586622,40.006058,172.590652,1091.150513,686.115173,...,38.984074,91.995575,29.020655,122.737984,9.439254,333.717194,41.528015,11.606652,39.238087,37.740280
2005-03-07,41.294605,262.363831,40.276272,6.985238,28.079178,27.608932,40.887211,173.788544,1098.723755,690.877197,...,39.012215,90.410088,29.423250,123.437881,9.968954,333.488220,42.797733,12.043824,39.336170,37.441586
2005-03-08,41.753368,261.901306,40.683311,7.359980,28.890419,27.924734,42.485378,170.508026,1077.983887,677.835754,...,38.963966,91.042397,29.551809,124.650925,9.868230,329.779785,49.497852,12.361642,39.096401,37.467564


In [379]:
symbol_to_sector = {}
for symbol in stock_list:
    ticker = yf.Ticker(symbol)
    info = ticker.info
    sector = info.get('sector', 'Unknown')
    symbol_to_sector[symbol] = sector

In [380]:
def create_features(df):
    df.reset_index(inplace=True)  
    df=df.melt(id_vars='Date', var_name='Symbol', value_name='Close')
    
    df['LMA'] =(df.groupby('Symbol')['Close'].transform(lambda x: x.shift(1).rolling(window=length*2).mean()))
    df['SMA'] = (df.groupby('Symbol')['Close'].transform(lambda x: x.shift(1).rolling(window=length).mean()))
    df['Relative_Position'] = (df['SMA']-df['LMA'])/df['LMA']
    df['ROC']=df.groupby('Symbol')['Close'].transform(lambda x: 100 * x.shift(1).pct_change(63))
    df['PCT']=df.groupby('Symbol')['Close'].transform(lambda x: (x.shift(1).pct_change(1)))
    df['Yearly_returns']=df.groupby('Symbol')['Close'].transform(lambda x: x.shift(1).pct_change(252))
    df['Support']=(df.groupby('Symbol')['Close'].transform(lambda x: x.shift(1).rolling(window=length).mean()))-2*(df.groupby('Symbol')['Close'].transform(lambda x: x.shift(1).rolling(window=length).std()))    
    df['Proximity_to_Support']=(df['Close'].shift(1)/df['Support']) -1
    df['Volatility'] = df.groupby('Symbol')['PCT'].transform(lambda x: x.shift(1).rolling(window=252).std())
    df['Sector'] = df['Symbol'].map(symbol_to_sector)
    
    return df

In [ ]:
df=create_features(df)
df=df.dropna()
train_df=df[df['Date']<split_date]
test_df=df[df['Date']>=split_date]

In [382]:
df.head()

,Date,Symbol,Close,LMA,SMA,Relative_Position,ROC,PCT,Yearly_returns,Support,Proximity_to_Support,Volatility,Sector
254,2006-03-08,ABB.NS,93.972099,73.849165,82.278073,0.114137,44.755822,0.028805,1.235818,75.492725,0.221338,0.018353,Industrials
255,2006-03-09,ABB.NS,97.345375,74.356753,82.896262,0.114845,50.683729,0.019196,1.267401,75.356602,0.247032,0.018418,Industrials
256,2006-03-10,ABB.NS,99.748741,74.917153,83.410860,0.113375,57.534507,0.035897,1.357339,74.223224,0.311522,0.018445,Industrials
257,2006-03-13,ABB.NS,99.347900,75.514613,83.972140,0.111999,60.256020,0.024689,1.388999,73.022623,0.365998,0.018553,Industrials
258,2006-03-14,ABB.NS,97.968895,76.116801,84.485969,0.109952,56.411949,-0.004019,1.445111,72.181171,0.376369,0.018594,Industrials


In [383]:
def Strategy1(df):
    output_df = pd.DataFrame(0.0, index=df[df['Symbol']==df.iloc[0]['Symbol']]['Date'], columns=stock_list)
    day_count=0
    for date in output_df.index:
        if day_count==0 or day_count==length:
            temp=df[df['Date']==date]
            bottom5_symbols = temp.nsmallest(5, 'Relative_Position')['Symbol']
            day_count=0

        for symbol in bottom5_symbols:
            output_df.loc[date,symbol] = 1/len(bottom5_symbols)

        day_count+=1
        
    return output_df

In [384]:
def Strategy2(df):
    output_df = pd.DataFrame(0.0, index=df[df['Symbol']==df.iloc[0]['Symbol']]['Date'],  columns=stock_list)
    day_count=0
    for date in output_df.index:
        if day_count==0 or day_count==length:
            temp=df[(df['Date']==date) & (df['ROC']>0)]
            top5_symbols = temp.nlargest(5, 'ROC')['Symbol']
            day_count=0
            
        for symbol in top5_symbols:
            output_df.loc[date,symbol] = 1/len(top5_symbols)
            
        day_count+=1
        
    return output_df

In [385]:
def Strategy3(df):
    output_df = pd.DataFrame(0.0, index=df[df['Symbol']==df.iloc[0]['Symbol']]['Date'], columns=stock_list)
    day_count=0
    for date in output_df.index:
        if day_count==0 or day_count==length:
            temp=df[df['Date']==date]
            bottom5_symbols = temp.nsmallest(5, 'Proximity_to_Support')['Symbol']
            day_count=0

        for symbol in bottom5_symbols:
            output_df.loc[date,symbol] = 1/len(bottom5_symbols)

        day_count+=1

    return output_df

In [ ]:
def Strategy4(df):
    output_df = pd.DataFrame(0.0, index=df[df['Symbol']==df.iloc[0]['Symbol']]['Date'], columns=stock_list)
    day_count=0
    for date in output_df.index:
        temp=df[(df['Date']==date) & (df['Yearly_returns']>risk_free_rate)]
        if day_count==0 or day_count==length:
            bottom5_symbols = temp.nsmallest(5, 'Volatility')['Symbol']
            day_count=0
        for symbol in bottom5_symbols:
            output_df.loc[date,symbol] = 1/len(bottom5_symbols)
        day_count+=1
    return output_df

In [387]:
def Strategy5(df):
    output_df = pd.DataFrame(0.0, index=df[df['Symbol'] == df.iloc[0]['Symbol']]['Date'], columns=stock_list)
    day_count=0
    for date in output_df.index:
        temp=df[df['Date'] == date]
        if day_count==0 or day_count==length:
            sector_avg_roc=temp.groupby('Sector')['ROC'].mean()
            if sector_avg_roc.empty or sector_avg_roc.isna().all():
                top_sector=None
            else:
                top_sector=sector_avg_roc.idxmax()
                
            sector_stocks=temp[temp['Sector']==top_sector]
            if sector_stocks.empty or sector_stocks['ROC'].isna().all():
                top5_symbols=None
            else:
                top5_symbols=sector_stocks.nlargest(5, 'ROC')['Symbol']

            day_count=0

        if top5_symbols is not None:
            for symbol in top5_symbols:
                output_df.loc[date, symbol] = 1 / len(top5_symbols)
        day_count+=1
    return output_df

In [388]:
weights_strategy_1=Strategy1(df)
weights_strategy_2=Strategy2(df)
weights_strategy_3=Strategy3(df)
weights_strategy_4=Strategy4(df)
weights_strategy_5=Strategy5(df)

test_weights_strategy_1=weights_strategy_1[weights_strategy_1.index>=test_df['Date'].iloc[0]]
train_weights_strategy_1=weights_strategy_1[weights_strategy_1.index<test_df['Date'].iloc[0]]
test_weights_strategy_2=weights_strategy_2[weights_strategy_2.index>=test_df['Date'].iloc[0]]
train_weights_strategy_2=weights_strategy_2[weights_strategy_2.index<test_df['Date'].iloc[0]]
test_weights_strategy_3=weights_strategy_3[weights_strategy_3.index>=test_df['Date'].iloc[0]]
train_weights_strategy_3=weights_strategy_3[weights_strategy_3.index<test_df['Date'].iloc[0]]
test_weights_strategy_4=weights_strategy_4[weights_strategy_4.index>=test_df['Date'].iloc[0]]
train_weights_strategy_4=weights_strategy_4[weights_strategy_4.index<test_df['Date'].iloc[0]]
test_weights_strategy_5=weights_strategy_5[weights_strategy_5.index>=test_df['Date'].iloc[0]]
train_weights_strategy_5=weights_strategy_5[weights_strategy_5.index<test_df['Date'].iloc[0]]

train_weights={
    "strategy_1":train_weights_strategy_1,
    "strategy_2":train_weights_strategy_2,
    "strategy_3":train_weights_strategy_3,
    "strategy_4":train_weights_strategy_4,
    "strategy_5":train_weights_strategy_5

}

test_weights={
    "strategy_1":test_weights_strategy_1,
    "strategy_2":test_weights_strategy_2,
    "strategy_3":test_weights_strategy_3,
    "strategy_4":test_weights_strategy_4,
    "strategy_5":test_weights_strategy_5

}

weights={
    "strategy_1":weights_strategy_1,
    "strategy_2":weights_strategy_2,
    "strategy_3":weights_strategy_3,
    "strategy_4":weights_strategy_4,
    "strategy_5":weights_strategy_5
}

In [389]:
def calc_returns(weights_df,df):
    weights_df = weights_df.fillna(0)
    returns_df= pd.DataFrame(index=weights_df.index)
    for stock in stock_list:
        returns_df=pd.concat([returns_df, df[df['Symbol']==stock].set_index('Date')[['Close']].rename(columns={'Close': stock}) .pct_change()], axis=1, join='inner')
                                                       
    returns_df = returns_df.fillna(0)
    returns_df = weights_df.mul(returns_df)
    
    return returns_df

In [390]:
def maketrainingdataset(weights_dict,df):
    returns_strategy_1=pd.DataFrame(calc_returns(weights_dict["strategy_1"],df).sum(axis=1),columns=['strategy_1'])
    returns_strategy_2=pd.DataFrame(calc_returns(weights_dict["strategy_2"],df).sum(axis=1),columns=['strategy_2'])
    returns_strategy_3=pd.DataFrame(calc_returns(weights_dict["strategy_3"],df).sum(axis=1),columns=['strategy_3'])
    returns_strategy_4=pd.DataFrame(calc_returns(weights_dict["strategy_4"],df).sum(axis=1),columns=['strategy_4'])
    returns_strategy_5=pd.DataFrame(calc_returns(weights_dict["strategy_5"],df).sum(axis=1),columns=['strategy_5'])
    
    returns_all_strategies = pd.concat([
        returns_strategy_1,
        returns_strategy_2,
        returns_strategy_3,
        returns_strategy_4,
        returns_strategy_5
    ], axis=1)
    returns_all_strategies=returns_all_strategies[1:]

    
    for w in [length,length*2]:
        for i in range(1,6):
            returns_all_strategies[f'S{i}_R{w}']= returns_all_strategies[f"strategy_{i}"].transform(lambda x: x.shift(1).rolling(window=w,min_periods=1).mean()) # mean returns of each strategy over past w days
            returns_all_strategies[f'S{i}_NR{w}'] = (1 + returns_all_strategies[f'strategy_{i}']).shift(1).rolling(window=w,min_periods=1).apply(lambda x: x.prod() - 1) # net return of each strategy over past w days
            returns_all_strategies[f'S{i}_V{w}']= (returns_all_strategies[f"strategy_{i}"].transform(lambda x: x.shift(1).rolling(window=w,min_periods=1).std())) # volatility of returns of each strategy over past w days
            returns_all_strategies[f'S{i}_S{w}']= ((returns_all_strategies[f'S{i}_R{w}']-daily_rate)/returns_all_strategies[f'S{i}_V{w}'].replace(0, 1e6)) # sharpe ratio of returns of each strategy over past w days / sqrt(252)
    

    
    next_returns = returns_all_strategies[[f'S1_NR{length}',f'S2_NR{length}',f'S3_NR{length}',f'S4_NR{length}',f'S5_NR{length}']].shift(-length)
    next_returns.dropna(inplace=True)
    best_strategy = next_returns.idxmax(axis=1)
    best_strategy.dropna(inplace=True)
    best_strategy = best_strategy.str.extract(r'(\d+)').astype(int)[0]-1
    returns_all_strategies=returns_all_strategies.iloc[:-1]
    
    return returns_all_strategies,best_strategy

In [ ]:
x,y=maketrainingdataset(weights,df)
x=x[x.index<=y.index[-1]]
x = x.replace([np.inf, -np.inf], 0)
x.dropna(inplace=True)
y=y[y.index>=x.index[0]]
x_train=x[x.index<test_df['Date'].iloc[0]]
y_train=y[y.index<test_df['Date'].iloc[0]]
x_test=x[x.index>=test_df['Date'].iloc[0]]
y_test=y[y.index>=test_df['Date'].iloc[0]]

In [392]:
x_train.head()

,strategy_1,strategy_2,strategy_3,strategy_4,strategy_5,S1_R30,S1_NR30,S1_V30,S1_S30,S2_R30,...,S3_V60,S3_S60,S4_R60,S4_NR60,S4_V60,S4_S60,S5_R60,S5_NR60,S5_V60,S5_S60
Date,,,,,,,,,,,,,,,,,,,,,
2006-03-13,-0.006581,-0.002332,0.000458,0.003977,-0.002091,0.000435,0.000836,0.008190,0.020301,0.017775,...,0.000280,8.180337,0.014285,0.028774,0.000057,247.660978,0.017775,0.035859,0.003814,4.589825
2006-03-14,-0.001615,0.004320,0.001396,-0.000219,0.008352,-0.001904,-0.005750,0.007067,-0.307367,0.011073,...,0.001229,1.294102,0.010849,0.032866,0.005952,1.777779,0.011153,0.033692,0.011783,0.923775
2006-03-16,-0.001181,0.010954,0.000362,-0.004748,0.001629,-0.001832,-0.007356,0.005772,-0.363827,0.009384,...,0.001030,1.432063,0.008082,0.032639,0.007365,1.060902,0.010453,0.042326,0.009722,1.047575
2006-03-17,-0.011456,0.002064,-0.007442,-0.002157,-0.013094,-0.001701,-0.008528,0.005007,-0.393405,0.009698,...,0.001085,1.104706,0.005516,0.027736,0.008579,0.611652,0.008688,0.044024,0.009298,0.905494
2006-03-20,-0.000751,-0.003541,0.002467,0.006615,0.007158,-0.003327,-0.019886,0.005993,-0.599968,0.008426,...,0.003765,-0.076032,0.004237,0.025520,0.008288,0.478830,0.005058,0.030354,0.012175,0.393349


In [393]:
y_train.head()

Date
2006-03-13    1
2006-03-14    1
2006-03-16    0
2006-03-17    0
2006-03-20    0
Name: 0, dtype: int32

In [394]:
param_grid = {
    'n_estimators': [100,300],
    'max_depth': [10,15,20],
    'min_samples_split': [50],
    'min_samples_leaf': [25],
    'max_features': ["sqrt"],
    'class_weight': ["balanced"],
    'random_state' : [42]
}

rf = RandomForestClassifier()

grid_search= GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=3,
    n_jobs=1
)

grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

model = grid_search.best_estimator_

model.fit(x_train,y_train)
y_pred = model.predict(x_test)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END class_weight=balanced, max_depth=10, max_features=sqrt, min_samples_leaf=25, min_samples_split=50, n_estimators=100, random_state=42;, score=0.179 total time=   2.3s
[CV 2/5] END class_weight=balanced, max_depth=10, max_features=sqrt, min_samples_leaf=25, min_samples_split=50, n_estimators=100, random_state=42;, score=0.137 total time=   2.2s
[CV 3/5] END class_weight=balanced, max_depth=10, max_features=sqrt, min_samples_leaf=25, min_samples_split=50, n_estimators=100, random_state=42;, score=0.130 total time=   2.6s
[CV 4/5] END class_weight=balanced, max_depth=10, max_features=sqrt, min_samples_leaf=25, min_samples_split=50, n_estimators=100, random_state=42;, score=0.172 total time=   2.3s
[CV 5/5] END class_weight=balanced, max_depth=10, max_features=sqrt, min_samples_leaf=25, min_samples_split=50, n_estimators=100, random_state=42;, score=0.221 total time=   2.3s
[CV 1/5] END class_weight=balanced, max_depth

In [395]:
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy of Random Forest: {100 * acc:.2f}%")

Accuracy of Random Forest: 25.00%


In [396]:
np.bincount(y_pred) # number occurences of each strategy from 1 to 5 in y_pred

array([364, 225, 239, 243, 381], dtype=int64)

In [397]:
np.set_printoptions(suppress=True)
importance_per_feature=model.feature_importances_
features_importance_df=pd.DataFrame({
    'feature':x_train.columns,
    'Importance':importance_per_feature
})

In [398]:
features_importance_df.head()


,feature,Importance
0,strategy_1,0.001449
1,strategy_2,0.001355
2,strategy_3,0.001158
3,strategy_4,0.001229
4,strategy_5,0.000987


In [399]:
def Strategy_rf(df,test_weights,y_pred):

    strategy_map = {0: 'strategy_1', 1: 'strategy_2', 2: 'strategy_3', 3: 'strategy_4' , 4: 'strategy_5'}
    pred_series = pd.Series(y_pred, index=x_test.index)
    selected_columns = pred_series.map(strategy_map)
    selected_columns.dropna(inplace=True)
    pred_df=selected_columns.to_frame(name='strategy')
    output_df = pd.DataFrame(0.0, index=df[df['Symbol']==df.iloc[0]['Symbol']]['Date'],  columns=stock_list)

    day_count=0

    for date in df[df['Symbol']==df.iloc[0]['Symbol']]['Date']:
        if date in pred_df.index:
            if day_count==0 or day_count==length:
                strategy = pred_df.loc[date, 'strategy']
                day_count=0
            day_count+=1
        else:
            strategy="strategy_1"
        output_df.loc[date] = test_weights[strategy].loc[date]
        
    return output_df

In [400]:
#bactester without transaction costs
def backtester_without_TC(weights_df, df):
    weights_df = weights_df.fillna(0) 
    returns_df = pd.DataFrame(index=weights_df.index)
    
    for stock in stock_list:
        stock_data = df[df['Symbol']==stock].set_index('Date')
        returns_df[stock] = stock_data['Close'].pct_change()
    
    returns_df = returns_df.fillna(0)
    returns_df = weights_df.mul(returns_df)
    portfolio_returns=returns_df.sum(axis=1)
    
    notional = 1
    portfolio_values = []
        
    for i, date in enumerate(weights_df.index):
        prev_weights = weights_df.shift(1).loc[date].fillna(0)
        notional *= (1 + portfolio_returns[date])
        portfolio_values.append(notional)
    
    cumulative_returns = pd.Series(portfolio_values)
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns - peak) / peak.replace(0, 1e-10)
    max_drawdown = drawdown.min() * 100
    
    start_date = df['Date'].iloc[0]
    end_date = df['Date'].iloc[-1]
    time_diff = relativedelta(end_date, start_date)
    years = time_diff.years + time_diff.months / 12 + time_diff.days / 252
    
    cagr = (portfolio_values[-1]**(1/years) - 1) * 100
    
    daily_returns = cumulative_returns.pct_change().dropna()
    sharpe = (daily_returns.mean() - daily_rate) / daily_returns.std() * np.sqrt(252)
    
    return [cagr, sharpe, max_drawdown]


In [401]:
def test_strategy_without_TC(df):

    weights_strategy_1=Strategy1(df)
    weights_strategy_2=Strategy2(df)
    weights_strategy_3=Strategy3(df)
    weights_strategy_4=Strategy4(df)
    weights_strategy_5=Strategy5(df)
    weights_strategy_rf=Strategy_rf(df,test_weights,y_pred)
    
    performanceStrategy1 = backtester_without_TC(weights_strategy_1,df)
    performanceStrategy2 = backtester_without_TC(weights_strategy_2,df)
    performanceStrategy3 = backtester_without_TC(weights_strategy_3,df)
    performanceStrategy4 = backtester_without_TC(weights_strategy_4,df)
    performanceStrategy5 = backtester_without_TC(weights_strategy_5,df)
    performanceStrategyrf = backtester_without_TC(weights_strategy_rf,df)

    output_df = pd.DataFrame({'Strategy1': performanceStrategy1, 'Strategy2': performanceStrategy2, 'Strategy3': performanceStrategy3, 'Strategy4': performanceStrategy4, 'Strategy5': performanceStrategy5, 'Strategyrf': performanceStrategyrf},index=['CAGR','Sharpe Ratio','Maximum_Drawdown'])
    return output_df

In [402]:
test_strategy_without_TC(test_df) 

,Strategy1,Strategy2,Strategy3,Strategy4,Strategy5,Strategyrf
CAGR,21.020384,13.010592,22.937357,14.856621,14.193334,32.233881
Sharpe Ratio,0.606402,0.357135,0.787486,0.530208,0.388370,0.944138
Max_Drawdown,-52.173210,-41.160684,-33.845369,-26.780908,-46.428266,-31.677657


In [403]:
#bactester with transaction costs
def backtester_with_TC(weights_df, df):
    weights_df = weights_df.fillna(0) 
    returns_df = pd.DataFrame(index=weights_df.index)
    
    for stock in stock_list:
        stock_data = df[df['Symbol']==stock].set_index('Date')
        returns_df[stock] = stock_data['Close'].pct_change()
    
    returns_df = returns_df.fillna(0)
    returns_df = weights_df.mul(returns_df)
    portfolio_returns=returns_df.sum(axis=1)
    
    notional = 1
    total_trades = 0  
    portfolio_values = []
    
    rebalance_days = weights_df.index[weights_df.ne(weights_df.shift(1)).any(axis=1)]
    
    for i, date in enumerate(weights_df.index):
        prev_weights = weights_df.shift(1).loc[date].fillna(0)
        if date in rebalance_days:
            prev_weights = weights_df.shift(1).loc[date] 
            turnover = (weights_df.loc[date] - prev_weights).abs().sum()
            tc = turnover * cost_rate
            notional *= (1 - tc) 
            trades_today = (weights_df.loc[date] != prev_weights).sum()
            total_trades += trades_today
            
        notional *= (1 + portfolio_returns[date])
        portfolio_values.append(notional)
    
    cumulative_returns = pd.Series(portfolio_values)
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns - peak) / peak.replace(0, 1e-10)
    max_drawdown = drawdown.min() * 100
    
    start_date = df['Date'].iloc[0]
    end_date = df['Date'].iloc[-1]
    time_diff = relativedelta(end_date, start_date)
    years = time_diff.years + time_diff.months / 12 + time_diff.days / 252
    
    cagr = (portfolio_values[-1]**(1/years) - 1) * 100
    
    daily_returns = cumulative_returns.pct_change().dropna()
    sharpe = (daily_returns.mean() - daily_rate) / daily_returns.std() * np.sqrt(252)
    
    return [cagr, sharpe, max_drawdown, total_trades]


In [404]:
def test_strategy_with_TC(df):
    weights_strategy_1=Strategy1(df)
    weights_strategy_2=Strategy2(df)
    weights_strategy_3=Strategy3(df)
    weights_strategy_4=Strategy4(df)
    weights_strategy_5=Strategy5(df)
    weights_strategy_rf=Strategy_rf(df,test_weights,y_pred)

    performanceStrategy1 = backtester_with_TC(weights_strategy_1,df)
    performanceStrategy2 = backtester_with_TC(weights_strategy_2,df)
    performanceStrategy3 = backtester_with_TC(weights_strategy_3,df)
    performanceStrategy4 = backtester_with_TC(weights_strategy_4,df)
    performanceStrategy5 = backtester_with_TC(weights_strategy_5,df)
    performanceStrategyrf = backtester_with_TC(weights_strategy_rf,df)

    output_df = pd.DataFrame({'Strategy1': performanceStrategy1,'Strategy2': performanceStrategy2, 'Strategy3': performanceStrategy3, 'Strategy4': performanceStrategy4, 'Strategy5': performanceStrategy5, 'Strategyrf': performanceStrategyrf},index=['CAGR','Sharpe Ratio','Maximum_Drowdown','Total Trades'])
    output_df.iloc[3] = output_df.iloc[3].astype(int)
    return output_df

In [405]:
test_strategy_with_TC(test_df)

,Strategy1,Strategy2,Strategy3,Strategy4,Strategy5,Strategyrf
CAGR,17.905704,10.542815,19.279703,13.460408,11.298060,26.301396
Sharpe Ratio,0.508155,0.265958,0.640535,0.454655,0.290514,0.769729
Max_Drowdown,-52.440734,-42.281511,-34.109987,-26.780908,-47.701199,-33.708700
Total Trades,463.000000,403.000000,525.000000,255.000000,406.000000,731.000000
